In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, confusion_matrix, accuracy_score
)
import pickle
import datetime
import numpy as np


In [ ]:
user = "erpnext_sql"
password = "rtp[ps4XRF0_pX"
db = "_abde597b15b69fc8"
host = "erpnext.responsible-ai.net"
port = "3333"

In [3]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}")

### Dateset

In [ ]:
data_sql_full = pd.read_sql("""
    Select 		
	po.name, 
        po.supplier, 
        po.total_qty,
        po.transaction_date as order_date,
        po_items.schedule_date,
        po_receipt.posting_date as delivered_date
    FROM `tabPurchase Order` po
    LEFT JOIN `tabPurchase Order Item` po_items
    	on po.name = po_items.parent
    LEFT JOIN `tabPurchase Receipt Item` po_receipt_item
    	on po.name = po_receipt_item.purchase_order 
    LEFT JOIN `tabPurchase Receipt` po_receipt
    	on po_receipt_item.parent = po_receipt.name
    """, engine)

In [7]:
data_sql_full.head()

,name,supplier,total_qty,transaction_date,set_warehouse,item_name,schedule_date,rate,quantity_ordered,warehouse,receipt_item_parent,delivered_date
0,PUR-ORD-2024-08-00001,Beans Inc.,750.0,2020-06-14,Naples - RR,Excelsa,2020-06-25,30.0,750.0,Naples - RR,MAT-PRE-2024-00001,2020-06-25
1,PUR-ORD-2024-08-00002,Beans Inc.,750.0,2023-04-24,Naples - RR,Excelsa,2023-04-29,30.0,750.0,Naples - RR,MAT-PRE-2024-00002,2023-05-04
2,PUR-ORD-2024-08-00003,Fair Trade AG,750.0,2024-09-08,Nairobi - RR,Arabica,2024-09-21,24.0,750.0,Nairobi - RR,None,None
3,PUR-ORD-2024-08-00004,Aromatico,1000.0,2022-10-26,Amsterdam - RR,Maragogype,2022-11-10,40.0,1000.0,Amsterdam - RR,MAT-PRE-2024-00003,2022-11-29
4,PUR-ORD-2024-08-00005,Aromatico,500.0,2022-03-18,London - RR,Maragogype Type B,2022-04-05,45.0,500.0,London - RR,MAT-PRE-2024-00004,2022-04-24


In [8]:
# Get size of dataset
data_sql_full.shape


(17181, 12)

In [9]:
# Check datatypes
data_sql_full.dtypes


name                    object
supplier                object
total_qty              float64
transaction_date        object
set_warehouse           object
item_name               object
schedule_date           object
rate                   float64
quantity_ordered       float64
warehouse               object
receipt_item_parent     object
delivered_date          object
dtype: object

In [10]:
# Drop nulls
data_sql_full.dropna(inplace=True, subset=["delivered_date", "schedule_date"])

In [11]:
# Convert timestamps
data_sql_full["delivered_date"] = pd.to_datetime(data_sql_full["delivered_date"])
data_sql_full["schedule_date"] = pd.to_datetime(data_sql_full["schedule_date"])

Calculate if purchase order was late

In [ ]:
data_sql_full["late"] = data_sql_full["delivered_date"] > data_sql_full["schedule_date"]


Calculate days late

In [ ]:

data_sql_full["days_late"] = (data_sql_full["delivered_date"] - data_sql_full["schedule_date"]).dt.days


Get average numbers of days late

In [ ]:


data_sql_full["days_late"].mean()


np.float64(10.406653394130114)

In [14]:


# Check days late by supplier
data_sql_full.groupby('supplier')["days_late"].mean().sort_values(ascending=False)


In [ ]:

# Create Dummies for Supplier
d_suppliers = pd.get_dummies(data_sql_full['supplier'], prefix="d")


In [ ]:

# Add dummies to dataset
data_for_model = pd.concat([data_sql_full, d_suppliers], axis=1)


In [ ]:

# Create feature list
features = ["total_qty"] + list(d_suppliers.columns)


## Train / Test split

In [ ]:
data_for_model["order_date"].max()
data_for_model["order_date"].min()


In [ ]:
# Check if first of 1. January 2024 is a good split date
data_for_model[data_for_model["order_date"] > datetime.date(2024, 1, 1)].shape[0] / data_for_model.shape[0]


In [ ]:

# Find better split date
data_for_model[data_for_model["order_date"] > datetime.date(2023, 3, 1)].shape[0] / data_for_model.shape[0]




In [ ]:
# Define Train dataset 
train = data_for_model[data_for_model["order_date"] <= datetime.date(2023, 3, 1)]


In [ ]:
# Define Test dataset
test = data_for_model[data_for_model["order_date"] > datetime.date(2023, 3, 1)]


## 1. Classifier Model

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=1234)


In [ ]:
rf_classifier.fit(train[features], train["late"])

In [ ]:
predictions = rf_classifier.predict(test[features])

In [ ]:
accuracy = accuracy_score(test["late"], predictions)

In [ ]:
accuracy

In [ ]:
cm = confusion_matrix(test["late"], predictions, labels=[True, False])

In [ ]:
cm

### Export Model

In [ ]:

export_name ="rf_1000_classifier.pkl"
with open(export_name, "wb") as file:
    pickle.dump(rf_classifier, file)

## 2 Model for predicting days late

In [ ]:

# Init Random Forest
rf = RandomForestRegressor(n_estimators=1000, random_state=1234)


In [ ]:

# Train model
rf.fit(train[features], train["days_late"])


In [ ]:


# Predict
predictions = rf.predict(test[features])


In [ ]:

# Calculate Mean Squared Error
mse = mean_squared_error(test["days_late"], predictions)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")



In [ ]:
# Mean absolute error
mae = mean_absolute_error(test["days_late"], predictions)
print(f"MAE: {mae}")


### Gradient boosted treess

In [ ]:
# Test gradient boosted trees
gbm = GradientBoostingRegressor(n_estimators=500, random_state=1234)

In [ ]:
gbm.fit(train[features], train["days_late"])

In [ ]:
predictions_gbm = gbm.predict(test[features])

In [ ]:
mse_gbm = mean_squared_error(test["days_late"], predictions_gbm)

In [ ]:
rmse_gbm = np.sqrt(mse_gbm)
print(f"RMSE: {rmse_gbm}")

In [ ]:
# Mean absolute error
mae_gbm = mean_absolute_error(test["days_late"], predictions_gbm)
print(f"MAE: {mae_gbm}")


In [ ]:
export_name ="gbm_500.pkl"
with open(export_name, "wb") as file:
    pickle.dump(gbm, file)